In [ ]:
%ls

drive/  sample_data/


In [1]:
%pwd

'/content'

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/projectrebel"

/content/drive/MyDrive/Colab Notebooks/projectrebel


In [ ]:
!unzip rebel.zip

Archive:  rebel.zip
   creating: rebel/
   creating: rebel/.idea/
  inflating: rebel/.idea/.gitignore  
   creating: rebel/.idea/inspectionProfiles/
  inflating: rebel/.idea/inspectionProfiles/profiles_settings.xml  
  inflating: rebel/.idea/inspectionProfiles/Project_Default.xml  
  inflating: rebel/.idea/misc.xml    
  inflating: rebel/.idea/modules.xml  
  inflating: rebel/.idea/rebel.iml   
  inflating: rebel/.idea/workspace.xml  
   creating: rebel/conf/
   creating: rebel/conf/data/
  inflating: rebel/conf/data/ade_data.yaml  
  inflating: rebel/conf/data/conll04_data.yaml  
  inflating: rebel/conf/data/default_data.yaml  
  inflating: rebel/conf/data/docred_data.yaml  
  inflating: rebel/conf/data/nyt_data.yaml  
  inflating: rebel/conf/data/tacred_data.yaml  
   creating: rebel/conf/model/
  inflating: rebel/conf/model/default_model.yaml  
  inflating: rebel/conf/model/rebel_model.yaml  
  inflating: rebel/conf/root.yaml    
   creating: rebel/conf/train/
  inflating: rebel/con

In [ ]:
%cd rebel/src

/content/drive/MyDrive/Colab Notebooks/projectrebel/rebel/src


In [ ]:
!python -m pip install pytorch-lightning==1.1.7
!python -m pip install hydra-core
!python -m pip install numpy==1.26.4
!python -m pip install datasets==2.14.6
!python -m pip install fsspec==2023.9.2
!python -m pip install gcsfs==2023.9.2

In [ ]:
!python train.py model=rebel_model data=conll04_data train=conll04_train

2025-06-10 09:43:16.803022: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749548597.051128    2008 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749548597.125968    2008 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-10 09:43:17.649019: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/content/drive/MyDrive/Colab Notebooks/projectrebel/rebel/src/train.py:107: UserWarning: 
The version_base parameter is not s

In [ ]:
!python train.py model=rebel_model data=nyt_data train=nyt_train

2025-06-11 07:02:06.719180: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749625326.979610    2230 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749625327.052107    2230 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-11 07:02:07.621891: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/content/drive/MyDrive/Colab Notebooks/projectrebel/rebel/src/train.py:107: UserWarning: 
The version_base parameter is not s

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(), 'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets


tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")
gen_kwargs = {
    "max_length": 256,
    "length_penalty": 0,
    "num_beams": 3,
    "num_return_sequences": 3,
}

# f = open('text.txt')
# raw_text = f.read()
# f.close()
raw_text = 'An art exhibit at the Hakawati/Theatre in Arab east Jerusalem was a series of portraits of Palestinians killed in the rebellion.\n'
'Israel television rejected a skit by comedian Tuvia/Tzafir that attacked public apathy by depicting an Israeli family watching TV while a fire raged outside.'

model_inputs = tokenizer(raw_text, max_length=256, padding=True, truncation=True, return_tensors='pt')

generated_tokens = model.generate(
    model_inputs["input_ids"].to(model.device),
    attention_mask=model_inputs["attention_mask"].to(model.device),
    **gen_kwargs,
)

decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

for idx, sentence in enumerate(decoded_preds):
    # print(f'Prediction triplets sentence {idx}')
    print(extract_triplets(sentence))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/344 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

[{'head': 'Hakawati/Theatre', 'type': 'located in the administrative territorial entity', 'tail': 'Jerusalem'}]
[{'head': 'Hakawati', 'type': 'located in the administrative territorial entity', 'tail': 'Jerusalem'}]
[{'head': 'Hakawati/Theatre', 'type': 'located in the administrative territorial entity', 'tail': 'Jerusalem'}, {'head': 'Arab east', 'type': 'located in the administrative territorial entity', 'tail': 'Jerusalem'}]
